In [1]:
# Load modules

import xarray as xr
import pandas as pd
import numpy as np
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import zarr
import gcsfs

# Load in data

In [2]:
cat = pd.read_csv('/data/keeling/a/cristi/a/esm_data/cmip6_catalog.csv')
cat

,activity_id,branch_method,branch_time_in_child,branch_time_in_parent,experiment,experiment_id,frequency,grid,grid_label,institution_id,...,standard_name,long_name,units,vertical_levels,init_year,start_time,end_time,time_range,path,version
0,CMIP,standard,0.0,0.0,abrupt quadrupling of CO2,abrupt-4xCO2,mon,native atmosphere N96 grid (144x192 latxlon),gn,CSIRO-ARCCSS,...,surface_upward_latent_heat_flux,Surface Upward Latent Heat Flux,W m-2,1.0,NaN,0950-01-16 12:00:00,1099-12-16 12:00:00,0950-01-16 12:00:00-1099-12-16 12:00:00,/data/cristi/a/cristi/esm_data/cmip6/ACCESS-CM...,v0
1,CMIP,standard,0.0,0.0,abrupt quadrupling of CO2,abrupt-4xCO2,mon,native atmosphere N96 grid (144x192 latxlon),gn,CSIRO-ARCCSS,...,surface_upward_sensible_heat_flux,Surface Upward Sensible Heat Flux,W m-2,1.0,NaN,0950-01-16 12:00:00,1099-12-16 12:00:00,0950-01-16 12:00:00-1099-12-16 12:00:00,/data/cristi/a/cristi/esm_data/cmip6/ACCESS-CM...,v0
2,CMIP,standard,0.0,0.0,abrupt quadrupling of CO2,abrupt-4xCO2,mon,native atmosphere N96 grid (144x192 latxlon),gn,CSIRO-ARCCSS,...,surface_downwelling_longwave_flux_in_air,Surface Downwelling Longwave Radiation,W m-2,1.0,NaN,0950-01-16 12:00:00,1099-12-16 12:00:00,0950-01-16 12:00:00-1099-12-16 12:00:00,/data/cristi/a/cristi/esm_data/cmip6/ACCESS-CM...,v0
3,CMIP,standard,0.0,0.0,abrupt quadrupling of CO2,abrupt-4xCO2,mon,native atmosphere N96 grid (144x192 latxlon),gn,CSIRO-ARCCSS,...,surface_upwelling_longwave_flux_in_air,Surface Upwelling Longwave Radiation,W m-2,1.0,NaN,0950-01-16 12:00:00,1099-12-16 12:00:00,0950-01-16 12:00:00-1099-12-16 12:00:00,/data/cristi/a/cristi/esm_data/cmip6/ACCESS-CM...,v0
4,CMIP,standard,0.0,0.0,abrupt quadrupling of CO2,abrupt-4xCO2,mon,native atmosphere N96 grid (144x192 latxlon),gn,CSIRO-ARCCSS,...,toa_outgoing_longwave_flux,TOA Outgoing Longwave Radiation,W m-2,1.0,NaN,0950-01-16 12:00:00,1099-12-16 12:00:00,0950-01-16 12:00:00-1099-12-16 12:00:00,/data/cristi/a/cristi/esm_data/cmip6/ACCESS-CM...,v0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11429,ScenarioMIP,standard,59400.0,59400.0,update of RCP8.5 based on SSP5,ssp585,mon,Native N96 grid; 192 x 144 longitude/latitude,gn,MOHC,...,surface_downwelling_shortwave_flux_in_air,Surface Downwelling Shortwave Radiation,W m-2,1.0,NaN,2015-01-16 00:00:00,2100-12-16 00:00:00,2015-01-16 00:00:00-2100-12-16 00:00:00,/data/cristi/a/cristi/esm_data/cmip6/UKESM1-0-...,v0
11430,ScenarioMIP,standard,59400.0,59400.0,update of RCP8.5 based on SSP5,ssp585,mon,Native N96 grid; 192 x 144 longitude/latitude,gn,MOHC,...,toa_incoming_shortwave_flux,TOA Incident Shortwave Radiation,W m-2,1.0,NaN,2015-01-16 00:00:00,2100-12-16 00:00:00,2015-01-16 00:00:00-2100-12-16 00:00:00,/data/cristi/a/cristi/esm_data/cmip6/UKESM1-0-...,v0
11431,ScenarioMIP,standard,59400.0,59400.0,update of RCP8.5 based on SSP5,ssp585,mon,Native N96 grid; 192 x 144 longitude/latitude,gn,MOHC,...,surface_upwelling_shortwave_flux_in_air,Surface Upwelling Shortwave Radiation,W m-2,1.0,NaN,2015-01-16 00:00:00,2100-12-16 00:00:00,2015-01-16 00:00:00-2100-12-16 00:00:00,/data/cristi/a/cristi/esm_data/cmip6/UKESM1-0-...,v0
11432,ScenarioMIP,standard,59400.0,59400.0,update of RCP8.5 based on SSP5,ssp585,mon,Native N96 grid; 192 x 144 longitude/latitude,gn,MOHC,...,toa_outgoing_shortwave_flux,TOA Outgoing Shortwave Radiation,W m-2,1.0,NaN,2015-01-16 00:00:00,2100-12-16 00:00:00,2015-01-16 00:00:00-2100-12-16 00:00:00,/data/cristi/a/cristi/esm_data/cmip6/UKESM1-0-...,v0


In [9]:
path_rsdt_control = cat.loc[(cat['variable_id']=='rsdt') &
               (cat['activity_id']=='CMIP') &
               (cat['frequency']=='mon') & 
               (cat['experiment_id'].str.contains('Control'))].drop_duplicates('source_id').head(38)['path'].to_list()
# Not enough data
path_rsdt_control.remove('/data/cristi/a/cristi/esm_data/cmip6/IITM-ESM/piControl/r1i1p1f1/rsdt_Amon_IITM-ESM_piControl_r1i1p1f1_gn_192601-193512.nc')
print(path_rsdt_control)
print(len(path))

['/data/cristi/a/cristi/esm_data/cmip6/ACCESS-CM2/piControl/r1i1p1f1/rsdt_Amon_ACCESS-CM2_piControl_r1i1p1f1_gn_095001-144912.nc', '/data/cristi/a/cristi/esm_data/cmip6/ACCESS-ESM1-5/piControl/r1i1p1f1/rsdt_Amon_ACCESS-ESM1-5_piControl_r1i1p1f1_gn_010101-060012.nc', '/data/cristi/a/cristi/esm_data/cmip6/AWI-CM-1-1-MR/piControl/r1i1p1f1/rsdt_Amon_AWI-CM-1-1-MR_piControl_r1i1p1f1_gn_240101-290012.nc', '/data/cristi/a/cristi/esm_data/cmip6/AWI-ESM-1-1-LR/piControl/r1i1p1f1/rsdt_Amon_AWI-ESM-1-1-LR_piControl_r1i1p1f1_gn_185501-195412.nc', '/data/cristi/a/cristi/esm_data/cmip6/BCC-CSM2-MR/piControl/r1i1p1f1/rsdt_Amon_BCC-CSM2-MR_piControl_r1i1p1f1_gn_185001-244912.nc', '/data/cristi/a/cristi/esm_data/cmip6/BCC-ESM1/piControl/r1i1p1f1/rsdt_Amon_BCC-ESM1_piControl_r1i1p1f1_gn_185001-230012.nc', '/data/cristi/a/cristi/esm_data/cmip6/CAMS-CSM1-0/piControl/r1i1p1f1/rsdt_Amon_CAMS-CSM1-0_piControl_r1i1p1f1_gn_290001-339912.nc', '/data/cristi/a/cristi/esm_data/cmip6/CAS-ESM2-0/piControl/r1i1p1f1/r

In [11]:
new_lats=np.arange(-87,88,2)
new_lons=np.arange(1,359,2)

dset_list_rsdt_control = []

for file in path_rsdt_control:
    name = file.removeprefix('/data/cristi/a/cristi/esm_data/cmip6/')
    print(name)
    ds = xr.open_dataset(file)
    ds_rsdt_rolling = ds.rsdt.interp(lat=new_lats,lon=new_lons).rolling(time=300, center=True).mean('time')
    ds_rsdt = ds_rsdt_rolling.std('time')
    ds_rsdt['name'] = name # Creating a new dimension for each ensemble member
    dset_list_rsdt_control.append(ds_rsdt)

ACCESS-CM2/piControl/r1i1p1f1/rsdt_Amon_ACCESS-CM2_piControl_r1i1p1f1_gn_095001-144912.nc


/data/keeling/a/mailes2/miniconda3/envs/paleoproj/lib/python3.10/site-packages/xarray/coding/times.py:688: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/data/keeling/a/mailes2/miniconda3/envs/paleoproj/lib/python3.10/site-packages/xarray/core/indexing.py:524: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return np.asarray(array[self.key], dtype=None)


ACCESS-ESM1-5/piControl/r1i1p1f1/rsdt_Amon_ACCESS-ESM1-5_piControl_r1i1p1f1_gn_010101-060012.nc


/data/keeling/a/mailes2/miniconda3/envs/paleoproj/lib/python3.10/site-packages/xarray/coding/times.py:150: SerializationWarning: Ambiguous reference date string: 101-01-01. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0101-01-01). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)
/data/keeling/a/mailes2/miniconda3/envs/paleoproj/lib/python3.10/site-packages/xarray/coding/times.py:688: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/data/keeling/a/mailes2/miniconda3/envs/paleoproj/lib/python3.10/site-packages/xarray/core/indexing.py:524: SerializationWarning: Unable to decode time axis into full numpy.datetime64 ob

AWI-CM-1-1-MR/piControl/r1i1p1f1/rsdt_Amon_AWI-CM-1-1-MR_piControl_r1i1p1f1_gn_240101-290012.nc


...

/data/keeling/a/mailes2/miniconda3/envs/paleoproj/lib/python3.10/site-packages/xarray/coding/times.py:688: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)


EC-Earth3/piControl/r1i1p1f1/rsdt_Amon_EC-Earth3_piControl_r1i1p1f1_gr_225901-275912.nc


/data/keeling/a/mailes2/miniconda3/envs/paleoproj/lib/python3.10/site-packages/xarray/core/indexing.py:524: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return np.asarray(array[self.key], dtype=None)
/data/keeling/a/mailes2/miniconda3/envs/paleoproj/lib/python3.10/site-packages/xarray/coding/times.py:688: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/data/keeling/a/mailes2/miniconda3/envs/paleoproj/lib/python3.10/site-packages/xarray/core/indexing.py:524: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return np.asarray(array[self.key], dtype=None)


FGOALS-f3-L/piControl/r1i1p1f1/rsdt_Amon_FGOALS-f3-L_piControl_r1i1p1f1_gr_060001-116012.nc
FGOALS-g3/piControl/r1i1p1f1/rsdt_Amon_FGOALS-g3_piControl_r1i1p1f1_gn_020001-089912.nc
FIO-ESM-2-0/piControl/r1i1p1f1/rsdt_Amon_FIO-ESM-2-0_piControl_r1i1p1f1_gn_040101-090012.nc
GISS-E2-1-G-CC/piControl/r1i1p1f1/rsdt_Amon_GISS-E2-1-G-CC_piControl_r1i1p1f1_gn_185001-201412.nc
GISS-E2-1-G/piControl/r101i1p1f1/rsdt_Amon_GISS-E2-1-G_piControl_r101i1p1f1_gn_185001-201412.nc
GISS-E2-1-H/piControl/r1i1p1f1/rsdt_Amon_GISS-E2-1-H_piControl_r1i1p1f1_gn_318001-323012.nc
GISS-E2-2-G/piControl/r1i1p1f1/rsdt_Amon_GISS-E2-2-G_piControl_r1i1p1f1_gn_200001-202512.nc
HadGEM3-GC31-LL/piControl/r1i1p1f1/rsdt_Amon_HadGEM3-GC31-LL_piControl_r1i1p1f1_gn_185001-234912.nc
INM-CM4-8/piControl/r1i1p1f1/rsdt_Amon_INM-CM4-8_piControl_r1i1p1f1_gr1_185001-238012.nc
INM-CM5-0/piControl/r1i1p1f1/rsdt_Amon_INM-CM5-0_piControl_r1i1p1f1_gr1_199601-319612.nc
IPSL-CM6A-LR/piControl/r1i1p1f1/rsdt_Amon_IPSL-CM6A-LR_piControl_r1i1p1f

/data/keeling/a/mailes2/miniconda3/envs/paleoproj/lib/python3.10/site-packages/xarray/coding/times.py:688: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/data/keeling/a/mailes2/miniconda3/envs/paleoproj/lib/python3.10/site-packages/xarray/core/indexing.py:524: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return np.asarray(array[self.key], dtype=None)


In [14]:
control_rsdt = xr.concat(dset_list_rsdt_control, dim='name',coords='minimal', compat='override')
control_rsdt

<xarray.DataArray 'rsdt' (name: 37, lat: 88, lon: 179)>
array([[[2.82129509e-02, 2.82129509e-02, 2.82129509e-02, ...,
         2.82129509e-02, 2.82129509e-02, 2.82129509e-02],
        [2.80937254e-02, 2.80937254e-02, 2.80937254e-02, ...,
         2.80937254e-02, 2.80937254e-02, 2.80937254e-02],
        [2.78710275e-02, 2.78710275e-02, 2.78710275e-02, ...,
         2.78710275e-02, 2.78710275e-02, 2.78710275e-02],
        ...,
        [2.01180679e-02, 2.01180679e-02, 2.01180679e-02, ...,
         2.01180679e-02, 2.01180679e-02, 2.01180679e-02],
        [2.01797548e-02, 2.01797548e-02, 2.01797548e-02, ...,
         2.01797548e-02, 2.01797548e-02, 2.01797548e-02],
        [2.02323512e-02, 2.02323512e-02, 2.02323512e-02, ...,
         2.02323512e-02, 2.02323512e-02, 2.02323512e-02]],

       [[2.79233448e-02, 2.79233448e-02, 2.79233448e-02, ...,
         2.79233448e-02, 2.79233448e-02, 2.79233448e-02],
        [2.78149161e-02, 2.78149161e-02, 2.78149161e-02, ...,
         2.78149161e-02, 2.78149161e-02, 2.78149161e-02],
        [2.75817746e-02, 2.75817746e-02, 2.75817746e-02, ...,
         2.75817746e-02, 2.75817746e-02, 2.75817746e-02],
...
        [6.06681352e-07, 6.28399199e-07, 6.44128606e-07, ...,
         5.22622657e-07, 6.01644638e-07, 6.26995634e-07],
        [6.13664858e-07, 6.19924303e-07, 6.55325741e-07, ...,
         6.06445411e-07, 5.95751752e-07, 6.50781701e-07],
        [5.82899249e-07, 5.69524163e-07, 4.43609978e-07, ...,
         4.63029453e-07, 4.47143365e-07, 4.14072394e-07]],

       [[1.51877397e-02, 1.51876998e-02, 1.51877291e-02, ...,
         1.51878453e-02, 1.51878286e-02, 1.51878286e-02],
        [1.51344754e-02, 1.51343956e-02, 1.51343523e-02, ...,
         1.51346897e-02, 1.51346874e-02, 1.51346004e-02],
        [1.50413309e-02, 1.50411569e-02, 1.50409469e-02, ...,
         1.50420649e-02, 1.50418770e-02, 1.50416930e-02],
        ...,
        [4.19504451e-11, 1.06865648e-11, 4.99369435e-11, ...,
         4.55884239e-11, 5.11874967e-11, 3.62092578e-11],
        [2.92743587e-11, 1.86446414e-11, 9.15179044e-12, ...,
         1.49213995e-11, 2.30500063e-11, 1.59445770e-11],
        [3.89377419e-12, 3.29976046e-11, 3.66640072e-11, ...,
         1.33013600e-11, 1.03455022e-11, 3.40776296e-11]]])
Coordinates:
  * lat      (lat) int64 -87 -85 -83 -81 -79 -77 -75 ... 75 77 79 81 83 85 87
  * lon      (lon) int64 1 3 5 7 9 11 13 15 ... 343 345 347 349 351 353 355 357
  * name     (name) <U103 'ACCESS-CM2/piControl/r1i1p1f1/rsdt_Amon_ACCESS-CM2...